In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from src.layers.flow import get_mask, MaskedLinear

In [ ]:
num_samples = 100
t = torch.linspace(0, 4 * torch.pi, 200).float()
curves = []
labels = []
for _ in range(num_samples):

    a = torch.rand(1).item() * 0.5  # Initial radius
    b = torch.rand(1).item() * 0.2  # Growth rate

    direction = torch.randint(0, 2, (1,)).item()  # 0=clockwise, 1=ccw

    # theta = torch.linspace(0, 4 * torch.pi, self.seq_len)  # Two full rotations
    r = a + b * t
    if direction == 0:
        x = r * torch.cos(t)
        y = r * torch.sin(t)
    else:
        x = -r * torch.cos(t)
        y = r * torch.sin(t)

    x += torch.randn_like(x) * 0.01
    y += torch.randn_like(y) * 0.01

    curve = torch.stack([x, y], dim=1)
    curves.append(curve)
    labels.append(direction)
data, class_cond = torch.stack(curves), torch.tensor(labels)
print(data.shape)
print(class_cond.shape)

In [ ]:
x = torch.randn(1000, 200, 2)
mask = torch.ones(1000, 200, 2)
total_seq_len=mask.shape[1]
delta = 50
rand_start = torch.randint(0, total_seq_len-delta, (1000,))
print(rand_start[:10])
max_delta = 10
# delta = (torch.rand(1000, device=x.device) * (max_delta + 1)).long()
end = rand_start + delta
print(end[:10])

# 生成时间步索引并比较生成布尔掩码
t = torch.arange(200, device=x.device).view(1, -1)  # 形状 [1, 200]
mask_2d = (t >= rand_start.view(-1, 1)) & (t <= end.view(-1, 1))  # 形状 [1000, 200]
mask_3d = mask_2d.unsqueeze(-1).expand_as(mask)
(~mask_3d) * x
plt.plot(x[0, :, 0])
plt.plot(((~mask_3d) * x)[0, :, 0])

In [6]:
import torch
import numpy as np
import scipy
import torch.nn.functional as F
import matplotlib.pyplot as plt
from src.model.gan.psagan import ProGenerator, ProDiscriminator

pg = ProGenerator(
    seq_len=64,
    seq_dim=3,
    time_feat_dim=5,
    ks_conv=3,
    key_features=8,
    value_features=8,
    ks_value=1,
    ks_query=1,
    ks_key=1,
    # device="cpu",
)
# x = torch.randn((8, 3, 16))
# time_feat = torch.randn((8, 5, 16))
# pg(x, time_feat).shape


pd = ProDiscriminator(
    seq_len=64,
    seq_dim=3,
    time_feat_dim=5,
    ks_conv=3,
    key_features=8,
    value_features=8,
    ks_value=1,
    ks_query=1,
    ks_key=1,
    # device="cpu",
)
x = torch.randn((8, 3, 64))
time_feat = torch.randn((8, 5, 64))
print(pg(x, time_feat, depth=3).shape)
print(pd(x, time_feat).shape)



torch.Size([8, 37, 16])
torch.Size([8, 37, 32])
torch.Size([8, 37, 64])
torch.Size([8, 3, 64])
torch.Size([8, 1])


In [ ]:
import math
tl = 100
2 ** math.log2(tl).__ceil__() > tl